### Machine Learning in Production - Part II

#### Hug API

Like `Flask`, we'll be taking the same machine learning model to production using `hug` & `uwsgi`.

NOTE: Hug only has `python3` compatibility. Any `sklearn` models you save via `pickle` or `joblib` would give you problems when you try to open it via Hug.

For the very same reason, we'll be moving to `python3` to create an API.

Follow the steps:

- Create a virtual environment, `hug_api.yml` has been provided.
- Create `sklearn` model and serialize it:

In [1]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
boston = load_boston()

X_train, X_test, y_train, y_test = train_test_split(boston.data, boston.target, random_state=0)

In [2]:
import os
import json
import requests
from sklearn.externals import joblib
from sklearn.linear_model import Ridge
import numpy as np
import pandas as pd

In [4]:
ridge = Ridge()
ridge.fit(X_train,y_train)

np.savetxt("../data/X_test.csv", X_test, delimiter=",")

#Saving the model
pickled_model2 = 'model2.pk'
joblib.dump(ridge, os.getcwd()+'/'+str(pickled_model2))

['/Users/Satish/prato/hug_api/notebooks/model2.pk']

- For purposes of prediction, we'll be saving the `X_test` as a `.csv` file.

In [8]:
df = pd.read_csv('../data/X_test.csv', header=None)

In [9]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.06724,0.0,3.24,0.0,0.460,6.333,17.2,5.2146,4.0,430.0,16.9,375.21,7.34
1,9.23230,0.0,18.10,0.0,0.631,6.216,100.0,1.1691,24.0,666.0,20.2,366.15,9.53
2,0.11425,0.0,13.89,1.0,0.550,6.373,92.4,3.3633,5.0,276.0,16.4,393.74,10.50
3,24.80170,0.0,18.10,0.0,0.693,5.349,96.0,1.7028,24.0,666.0,20.2,396.90,19.77
4,0.05646,0.0,12.83,0.0,0.437,6.232,53.7,5.0141,5.0,398.0,18.7,386.40,12.34


In [19]:
df.shape

(127, 13)

- Fire up the terminal and run the `uwsgi` commands:
    * `uwsgi --http 0.0.0.0:8000 --wsgi-file server.py --master --processes 2 --callable __hug_wsgi__`
    
- Query the API using the command below:

In [78]:
header = {'Content-Type': 'application/json', \
                  'Accept': 'application/json'}

df = pd.read_csv('../data/X_test.csv')
data = df.to_json(orient='split')
resp = requests.post("http://0.0.0.0:8000/predict", \
                    data = json.dumps(data),\
                    headers= header)

In [81]:
print("The status code recieved is {}".format(resp.status_code))
print("\n")
print("The prediction values are:\n")
print(resp.content)

The status code recieved is 200


The prediction values are:

b'"{\\"predictions\\": \\"{\\\\\\"0\\\\\\":23.1970246692,\\\\\\"1\\\\\\":28.7831062489,\\\\\\"2\\\\\\":12.0389356585,\\\\\\"3\\\\\\":20.7348465382,\\\\\\"4\\\\\\":19.9706031476,\\\\\\"5\\\\\\":20.1416154764,\\\\\\"6\\\\\\":21.8972532839,\\\\\\"7\\\\\\":19.1367463674,\\\\\\"8\\\\\\":19.6964533114,\\\\\\"9\\\\\\":4.9026668722,\\\\\\"10\\\\\\":15.224867093,\\\\\\"11\\\\\\":17.3341126078,\\\\\\"12\\\\\\":5.3119671669,\\\\\\"13\\\\\\":39.4253347312,\\\\\\"14\\\\\\":32.2486492312,\\\\\\"15\\\\\\":21.6364944381,\\\\\\"16\\\\\\":36.226486366,\\\\\\"17\\\\\\":31.1512965416,\\\\\\"18\\\\\\":23.4883343239,\\\\\\"19\\\\\\":25.044188137,\\\\\\"20\\\\\\":23.8016226456,\\\\\\"21\\\\\\":20.5146124596,\\\\\\"22\\\\\\":30.2995961069,\\\\\\"23\\\\\\":22.3245598983,\\\\\\"24\\\\\\":9.286593858,\\\\\\"25\\\\\\":17.8754575572,\\\\\\"26\\\\\\":19.3292449293,\\\\\\"27\\\\\\":35.3485825098,\\\\\\"28\\\\\\":20.4004817178,\\\\\\"29\\\\\\":17.611791871